In [8]:
import numpy as np #Para trabajar con arreglos

In [9]:
# Construcción de grilla

class Mapa:

    def __init__(self,ancho,alto,inicio):
        self.ancho = ancho
        self.alto = alto
        self.fila = inicio[0]
        self.columna = inicio[1]

    def conjunto(self,recompensas,acciones):
        self.recompensas = recompensas # diccionario de posibles recompensas para cada estado (fila,columna) : r
        self.acciones = acciones # diccionario de posibles acciones para cada estado (fila,columna) : ('U','D','L','R')

    def fijar_estado(self,s):
        self.fila = s[0]
        self.columna = s[1]

    def estado_actual(self):
        return (self.fila,self.columna)

    def es_final(self,s):
        return s not in self.acciones

    def mover(self,accion):
        if accion in self.acciones[self.estado_actual()] :
            if accion == 'U':
                self.fila -= 1
            elif accion == 'D':
                self.fila += 1
            elif accion == 'L':
                self.columna -= 1
            elif accion == 'R':
                self.columna += 1
        return self.recompensas.get(self.estado_actual(),0)

    def deshacer(self,accion):
        if accion in self.acciones[self.estado_actual()] :
            if accion == 'U':
                self.fila += 1
            elif accion == 'D':
                self.fila -= 1
            elif accion == 'L':
                self.columna += 1
            elif accion == 'R':
                self.columna -= 1
        assert(self.estado_actual() in self.todos_estados())

    def fin(self):
        return self.estado_actual not in self.acciones

    def todos_estados(self):
        return set(list(self.acciones.keys()) + list(self.recompensas.keys()))

In [10]:
#CONFIGURACIÓN DEL PROBLEMA
# Definición de la función mapa_con_costo con costo por defecto de -0.1
def mapa_con_costo(costo_paso = -0.1):
    # Crea un objeto Mapa de 4 columnas (ancho), 3 filas (alto) y estado inicial en (2,0)
    g = Mapa(4,3,(2,0))
    # Define las recompensas fijas: +1 en (0,3) y -1 en (1,3)
    recompensas = {(0,3) : 1, (1,3):-1}
    # Diccionario que define las acciones posibles desde cada celda del mapa
    acciones = {
        (0,0) : ('D','R'),       # desde (0,0) se puede mover abajo o derecha
        (0,1) : ('L','R'),       # desde (0,1) se puede mover izquierda o derecha
        (0,2) : ('L','R','D'),   # desde (0,2) se puede mover izq, der o abajo
#        (0,3) : ('',),          # (comentado) sin acciones desde celda terminal (0,3)
        (1,0) : ('U','D'),       # desde (1,0) se puede mover arriba o abajo
#        (1,1) : ('',),          # (comentado) celda bloqueada (pared)
        (1,2) : ('U','D','R'),   # desde (1,2) se puede mover arriba, abajo o derecha
#        (1,3) : ('',),          # (comentado) sin acciones desde celda terminal (1,3)
        (2,0) : ('U','R'),       # desde (2,0) se puede mover arriba o derecha
        (2,1) : ('L','R'),       # desde (2,1) se puede mover izquierda o derecha
        (2,2) : ('U','L','R'),   # desde (2,2) se puede mover arriba, izquierda o derecha
        (2,3) : ('L','U')        # desde (2,3) se puede mover izquierda o arriba
        }
    # Asigna las recompensas y acciones iniciales al mapa
    g.conjunto(recompensas,acciones)
    # Reinicia recompensas como diccionario vacío
    recompensas = {}
    # Recorre todas las celdas del mapa por filas y columnas
    for fila in range(g.alto):
        for columna in range(g.ancho):
            # Asigna costo de paso (negativo) a todas las celdas
            recompensas[(fila,columna)] = costo_paso
    # Vuelve a asignar +1 en (0,3)
    recompensas[(0,3)] = 1
    # Vuelve a asignar -1 en (1,3)
    recompensas[(1,3)] = -1
    # Elimina la celda (1,1) de las recompensas (pared o inaccesible)
    recompensas.pop((1,1),None)
    # Actualiza el mapa con las recompensas de costo de paso y mismas acciones
    g.conjunto(recompensas,g.acciones)
    # Retorna el objeto mapa configurado
    return g


In [11]:
#Mostrar valores def mostrar_valores(V,g):
# Función para mostrar los valores de cada celda en el mapa
def mostrar_valores(V,g):
    # Recorre las filas del mapa
    for fila in range(g.alto):
        # Imprime una línea de separación
        print(20*"-")
        # Recorre las columnas del mapa
        for columna in range(g.ancho):
            # Obtiene el valor V de la celda (fila,columna), 0 si no existe
            v = V.get((fila,columna),0)

            # Imprime el valor con dos decimales en la celda
            print(" %.2f|" % v, end="")
        # Salto de línea al terminar la fila
        print("")

# Función para mostrar la política (acciones) de cada celda en el mapa
def mostrar_politica(P,g) :
    # Recorre las filas del mapa
    for fila in range(g.alto):
        # Imprime una línea de separación
        print(20*"-")
        # Recorre las columnas del mapa
        for columna in range(g.ancho):
            # Obtiene la acción asignada en la celda (fila,columna), vacío si no existe
            a = P.get((fila,columna),'')
            # Imprime la acción en la celda
            print(" %s |" % a, end="")
        # Salto de línea al terminar la fila
        print("")


In [12]:
# Número máximo de repeticiones de la iteración de valores
repeticiones = 20
# Contador de iteraciones
iteraciones = 0
# Factor de descuento (pondera el futuro)
gamma = 0.9
# Umbral de convergencia (tolerancia de error)
epsilon = 1e-3
# Conjunto de acciones posibles
total_acciones = ('U','D','L','R')
# Crea el mapa con costo de paso
mapa = mapa_con_costo()
# Obtiene todos los estados posibles del mapa
estados = mapa.todos_estados()
# Inicializa diccionario de valores
V = {}
# Asigna valor inicial 0.0 a todos los estados
for s in estados:
    V[s] = 0.0

# Bucle principal de Iteración de Valores
while True:
    # Reinicia delta (cambio máximo en valores)
    delta = 0
    # Incrementa el contador de iteraciones
    iteraciones += 1
    # Recorre todos los estados del mapa
    for s in estados:
        # Si el estado no tiene acciones (terminal o pared), lo salta
        if s not in mapa.acciones.keys():
            continue
        # Guarda el valor anterior del estado
        v = V[s]
        # Inicializa el máximo valor posible como -infinito
        max_val = float("-inf")
        # Evalúa cada acción posible en ese estado
        for accion in mapa.acciones[s]:
            # Coloca el mapa en el estado actual
            mapa.fijar_estado(s)
            # Ejecuta la acción y obtiene la recompensa
            r = mapa.mover(accion)
            # Calcula el valor: recompensa + valor futuro descontado
            val = r + gamma * V[mapa.estado_actual()]  # determinístico
            # Si este valor es mejor que el máximo actual, lo actualiza
            if val > max_val:
                max_val = val
        # Actualiza el valor del estado con el máximo valor hallado
        V[s] = max_val
        # Calcula la diferencia máxima entre valores viejos y nuevos
        delta = max(delta,abs(v-V[s]))

    # Condición de parada: convergencia (delta < epsilon) o repeticiones máximas
    if delta < epsilon or iteraciones >= repeticiones:
        # Muestra la tabla de valores finales
        mostrar_valores(V,mapa)
        # Termina el bucle
        break


--------------------
 0.62| 0.80| 1.00| 0.00|
--------------------
 0.46| 0.00| 0.80| 0.00|
--------------------
 0.31| 0.46| 0.62| 0.46|


In [13]:
# Diccionario para guardar la política (mejor acción por estado)
pi = {}
# Recorre todos los estados que tienen acciones posibles
for s in mapa.acciones.keys():
    # Inicializa el mejor valor como -infinito
    max_val = float("-inf")
    # Recorre todas las acciones posibles en el estado s
    for accion in mapa.acciones[s]:
        # Fija el estado actual en el mapa
        mapa.fijar_estado(s)
        # Ejecuta la acción y obtiene la recompensa inmediata
        r = mapa.mover(accion)
        # Calcula el valor: recompensa + valor del siguiente estado (greedy sin descuento aquí)
        val = r + V[mapa.estado_actual()]
        # Si este valor es mejor que el actual máximo, actualiza el máximo y la acción
        if val > max_val :
            max_val = val
            max_accion = accion
    # Asigna la mejor acción encontrada al estado en la política
    pi[s] = max_accion

# Muestra la política obtenida en formato de tabla
mostrar_politica(pi,mapa)


--------------------
 R | R | R |  |
--------------------
 U |  | U |  |
--------------------
 U | R | U | L |
